In [1]:
# Install AIF360 and required dependencies
!pip install -q aif360
!pip install -q 'aif360[all]'
!pip install -q pandas numpy scikit-learn matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 103.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.7/901.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Restart runtime (required for full functionality)
import os
os.kill(os.getpid(), 9)

In [1]:
# Mount Google drive and make current folder as default
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Set the current working directory to your desired folder in Google Drive
import os
os.chdir('/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main') # Replace 'Your_Folder_Name' with your folder name

In [3]:
from aif360.datasets import MEPSDataset19,AdultDataset
from aif360.datasets import StandardDataset, BinaryLabelDataset
from methods.mitigate_disparity import MultiLevelReweighing , BiasRemoverModel
from aif360.algorithms.preprocessing import Reweighing
# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from collections import defaultdict

import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from metrics.eval_metrics import print_metrics_binary


/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [4]:
random_seed = 57
# PYTHON
os.environ['PYTHONHASHSEED'] = str(random_seed)
random.seed(random_seed)

# NUMPY
np.random.seed(random_seed)

In [5]:
def test(dataset, model, thresh_arr,unprivileged_groups,privileged_groups):
    try:
        # sklearn classifier

        y_val_pred_prob = model.predict_proba(dataset.features)

        pos_ind = np.where(model.classes_ == dataset.favorable_label)[0][0]
    except AttributeError:
        # aif360 inprocessing algorithm
        y_val_pred_prob = model.predict(dataset).scores
        pos_ind = 0

    metric_arrs = defaultdict(list)

    for thresh in thresh_arr:

        y_val_pred = (y_val_pred_prob[:, pos_ind] > thresh).astype(np.float64)

        dataset_pred = dataset.copy()
        dataset_pred.labels = y_val_pred

        metric = ClassificationMetric(
                dataset, dataset_pred,
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
        acc_metrics_binary = print_metrics_binary(dataset.labels.ravel(),y_val_pred)

        metric_arrs['acc'].append(acc_metrics_binary['acc'])
        metric_arrs['auroc'].append(acc_metrics_binary['auroc'])
        metric_arrs['auprc'].append(acc_metrics_binary['auprc'])
        metric_arrs['bal_acc'].append((metric.true_positive_rate()
                                     + metric.true_negative_rate()) / 2)
        metric_arrs['avg_odds_diff'].append(metric.average_odds_difference())
        metric_arrs['disp_imp'].append(metric.disparate_impact())
        metric_arrs['stat_par_diff'].append(metric.statistical_parity_difference())
        metric_arrs['eq_opp_diff'].append(metric.equal_opportunity_difference())
        metric_arrs['eq_odd_diff'].append(metric.equalized_odds_difference())
        metric_arrs['theil_ind'].append(metric.theil_index())

    return metric_arrs

def describe_metrics(metrics, thresh_arr):
    best_ind = np.argmax(metrics['bal_acc'])
    print("Threshold corresponding to Best balanced accuracy: {:6.4f}".format(thresh_arr[best_ind]))
    print("Best balanced accuracy: {:6.4f}".format(metrics['bal_acc'][best_ind]))
    print("acc value: {:6.4f}".format(metrics['acc'][best_ind]))
    print("auroc value: {:6.4f}".format(metrics['auroc'][best_ind]))
    print("auprc value: {:6.4f}".format(metrics['auprc'][best_ind]))

    disp_imp_at_best_ind = min(metrics['disp_imp'][best_ind], 1/metrics['disp_imp'][best_ind])
    print("Corresponding min(DI, 1/DI) value: {:6.4f}".format(disp_imp_at_best_ind))
    print("Corresponding statistical parity difference value: {:6.4f}".format(metrics['stat_par_diff'][best_ind]))
    print("Corresponding equal opportunity difference value: {:6.4f}".format(metrics['eq_opp_diff'][best_ind]))
    print("Corresponding equal odds difference value: {:6.4f}".format(metrics['eq_odd_diff'][best_ind]))
    print("Corresponding average odds difference value: {:6.4f}".format(metrics['avg_odds_diff'][best_ind]))
    print("Corresponding Theil index value: {:6.4f}".format(metrics['theil_ind'][best_ind]))


## Old Codes

In [ ]:
# Load CSV
data_path = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Raw/Adult_Census_orig.csv"
df = pd.read_csv(data_path)

# Clean column names (strip + lowercase)
df.columns = df.columns.str.strip().str.lower()
# Ensure 'sex' and 'race' are numerical (0/1)
#df['sex'] = df['sex'].str.strip().str.lower().map({'male': 1, 'female': 0})
#df['race'] = df['race'].str.strip().str.lower().map(lambda x: 1 if x == 'white' else 0)

# Clean 'sex' and 'probability' column values
#df['probability'] = df['probability'].str.strip()

# Confirm value counts (optional check)
print("Sex values:", df['sex'].unique())
print("Label values:", df['income'].unique())
print('sex' in df.columns)
# Now pass to StandardDataset
dataset = StandardDataset(
    df=df,
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['sex', 'race'],       # <- multiple protected attributes
    privileged_classes=[[1], [1]],                   # <- list of privileged values
    instance_weights_name=None,
    categorical_features=[
        'workclass', 'education', 'marital-status',
        'occupation', 'relationship', 'native-country'
    ],
    features_to_keep=[],
    features_to_drop=[],
    na_values=['?']
)

# Output dimensions
print("Features shape:", dataset.features.shape)
print("Labels shape:", dataset.labels.shape)



Sex values: [1 0]
Label values: [0 1]
True
Features shape: (45222, 100)
Labels shape: (45222, 1)


In [ ]:
multi_privileged_groups = [
    {"feature_name": "race", "privileged_value": 1, "level": 1},
    {"feature_name": "sex", "privileged_value": 1, "level": 2},
]
multi_unprivileged_groups = [
    {"feature_name": "race", "unprivileged_value": 0, "level": 1},
    {"feature_name": "sex", "unprivileged_value": 0, "level": 2},
]
privileged_groups1 = [{"sex": 1}]
unprivileged_groups1 = [{"sex": 0}]
privileged_groups2 = [{"race": 1}]
unprivileged_groups2 = [{"race": 0}]

(dataset_orig_train,
 dataset_orig_val) = dataset.split([0.7], shuffle=True, seed=random_seed)

In [ ]:
ori_sex = BinaryLabelDatasetMetric(
    dataset=dataset, unprivileged_groups=unprivileged_groups1, privileged_groups=privileged_groups1)
ori_race = BinaryLabelDatasetMetric(
    dataset=dataset, unprivileged_groups=unprivileged_groups2, privileged_groups=privileged_groups2)

print(
    "before reweighing ,sex disparate impact and spd is "

    + str(ori_sex.disparate_impact())+" "
    + str(ori_sex.statistical_parity_difference())
)
print(
    "before reweighing ,race disparate impact and spd is "

    + str(ori_race.disparate_impact())+" "
    + str(ori_race.statistical_parity_difference())
)

model = make_pipeline(StandardScaler(),
                      LogisticRegression(solver='liblinear', random_state=random_seed))   # can add algorithms or modify pipeline as per your need
fit_params = {
    'logisticregression__sample_weight': dataset_orig_train.instance_weights}

lr_orig = model.fit(dataset_orig_train.features,
                    dataset_orig_train.labels.ravel(), **fit_params)
thresh_arr = np.linspace(0.01, 0.5, 50)

print("sex:")
ori_sex_val_metrics = test(dataset=dataset_orig_val,
                   model=lr_orig,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups1, privileged_groups=privileged_groups1)
lr_orig_best_ind = np.argmax(ori_sex_val_metrics['bal_acc'])
describe_metrics(ori_sex_val_metrics, thresh_arr)
print("race:")
ori_race_val_metrics = test(dataset=dataset_orig_val,
                   model=lr_orig,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups2, privileged_groups=privileged_groups2)
lr_orig_best_ind = np.argmax(ori_race_val_metrics['bal_acc'])
describe_metrics(ori_race_val_metrics, thresh_arr)


before reweighing ,sex disparate impact and spd is 0.3634695423643793 -0.198901432678815
before reweighing ,race disparate impact and spd is 0.6037688467181627 -0.10395937026830099
sex:
Threshold corresponding to Best balanced accuracy: 0.2200
Best balanced accuracy: 0.8161
acc value: 0.7912
auroc value: 0.8161
auprc value: 0.7259
Corresponding min(DI, 1/DI) value: 0.3071
Corresponding statistical parity difference value: -0.3494
Corresponding equal opportunity difference value: -0.1331
Corresponding equal odds difference value: 0.2498
Corresponding average odds difference value: -0.1915
Corresponding Theil index value: 0.0803
race:
Threshold corresponding to Best balanced accuracy: 0.2200
Best balanced accuracy: 0.8161
acc value: 0.7912
auroc value: 0.8161
auprc value: 0.7259
Corresponding min(DI, 1/DI) value: 0.5949
Corresponding statistical parity difference value: -0.1683
Corresponding equal opportunity difference value: -0.0569
Corresponding equal odds difference value: 0.1100
Cor

In [ ]:
# Using MultiLevel Reweighing Methods as proposed in M3Fair
mmrw = MultiLevelReweighing(multi_unprivileged_groups, multi_privileged_groups)
trans_adult_dataset = mmrw.fit(dataset).transform(dataset)
trans_adult_dataset_train = mmrw.fit(dataset_orig_train).transform(dataset_orig_train)

mmrw_trans_sex = BinaryLabelDatasetMetric(dataset=trans_adult_dataset,unprivileged_groups=unprivileged_groups1,privileged_groups=privileged_groups1)
mmrw_trans_race = BinaryLabelDatasetMetric(dataset=trans_adult_dataset,unprivileged_groups=unprivileged_groups2,privileged_groups=privileged_groups2)

print(
    "after reweighing ,sex disparate impact and spd are "

    + str(mmrw_trans_sex.disparate_impact())+" "
    + str(mmrw_trans_sex.statistical_parity_difference())
)

print(
    "after reweighing ,race disparate impact and spd are "

    + str(mmrw_trans_race.disparate_impact())+" "
    + str(mmrw_trans_race.statistical_parity_difference())
)

model = make_pipeline(StandardScaler(),
                      LogisticRegression(solver='liblinear', random_state=random_seed))
fit_params = {
    'logisticregression__sample_weight': trans_adult_dataset_train.instance_weights}

lr_mmrw = model.fit(trans_adult_dataset_train.features,
                    trans_adult_dataset_train.labels.ravel(), **fit_params)
thresh_arr = np.linspace(0.01, 0.5, 50)

print("sex:")
mmrw_sex_val_metrics = test(dataset=dataset_orig_val,
                   model=lr_mmrw,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups1, privileged_groups=privileged_groups1)

describe_metrics(mmrw_sex_val_metrics, thresh_arr)
print("race:")
mmrw_race_val_metrics = test(dataset=dataset_orig_val,
                   model=lr_mmrw,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups2, privileged_groups=privileged_groups2)

describe_metrics(mmrw_race_val_metrics, thresh_arr)

after reweighing ,sex disparate impact and spd are 1.0 0.0
after reweighing ,race disparate impact and spd are 0.9999999999999999 -2.7755575615628914e-17
sex:
Threshold corresponding to Best balanced accuracy: 0.2200
Best balanced accuracy: 0.8094
acc value: 0.7958
auroc value: 0.8094
auprc value: 0.7195
Corresponding min(DI, 1/DI) value: 0.5450
Corresponding statistical parity difference value: -0.1986
Corresponding equal opportunity difference value: 0.0627
Corresponding equal odds difference value: 0.1012
Corresponding average odds difference value: -0.0192
Corresponding Theil index value: 0.0862
race:
Threshold corresponding to Best balanced accuracy: 0.2200
Best balanced accuracy: 0.8094
acc value: 0.7958
auroc value: 0.8094
auprc value: 0.7195
Corresponding min(DI, 1/DI) value: 0.7653
Corresponding statistical parity difference value: -0.0904
Corresponding equal opportunity difference value: 0.0371
Corresponding equal odds difference value: 0.0371
Corresponding average odds diffe

In [ ]:
# Using Standard Reweighing Method in series - Sex first, followed by Race
rw_sex = Reweighing(unprivileged_groups=unprivileged_groups1,
                    privileged_groups=privileged_groups1)
rw_race = Reweighing(unprivileged_groups=unprivileged_groups2,
                     privileged_groups=privileged_groups2)

trans_sex_dataset = rw_sex.fit(dataset).transform(dataset)
trans_sex_race_dataset = rw_race.fit(
    trans_sex_dataset).transform(trans_sex_dataset)

trans_sex_dataset_train = rw_sex.fit(dataset_orig_train).transform(dataset_orig_train)
trans_sex_race_dataset_train = rw_race.fit(
    trans_sex_dataset_train).transform(trans_sex_dataset_train)

trans_sex_metric = BinaryLabelDatasetMetric(
    dataset=trans_sex_race_dataset, unprivileged_groups=unprivileged_groups1, privileged_groups=privileged_groups1)
trans_race_metric = BinaryLabelDatasetMetric(
    dataset=trans_sex_race_dataset, unprivileged_groups=unprivileged_groups2, privileged_groups=privileged_groups2)

print(
    "after reweighing ,sex disparate impact and spd are "

    + str(trans_sex_metric.disparate_impact())+" "
    + str(trans_sex_metric.statistical_parity_difference())
)


print(
    "after reweighing ,race disparate impact and spd are "

    + str(trans_race_metric.disparate_impact())+" "
    + str(trans_race_metric.statistical_parity_difference())
)

model = make_pipeline(StandardScaler(),
                      LogisticRegression(solver='liblinear', random_state=random_seed))
fit_params = {
    'logisticregression__sample_weight': trans_sex_race_dataset_train.instance_weights}

lr_rw = model.fit(trans_sex_race_dataset_train.features,
                    trans_sex_race_dataset_train.labels.ravel(), **fit_params)
thresh_arr = np.linspace(0.01, 0.5, 50)

print("sex:")
SeriesRW1_sex_val_metrics = test(dataset=dataset_orig_val,
                   model=lr_rw,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups1, privileged_groups=privileged_groups1)

describe_metrics(SeriesRW1_sex_val_metrics, thresh_arr)
print("race:")
SeriesRW1_race_val_metrics = test(dataset=dataset_orig_val,
                   model=lr_rw,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups2, privileged_groups=privileged_groups2)

describe_metrics(SeriesRW1_race_val_metrics, thresh_arr)



after reweighing ,sex disparate impact and spd are 1.0218649052280429 0.005380859115217462
after reweighing ,race disparate impact and spd are 0.9999999999999998 -5.551115123125783e-17
sex:
Threshold corresponding to Best balanced accuracy: 0.2200
Best balanced accuracy: 0.8091
acc value: 0.7960
auroc value: 0.8091
auprc value: 0.7191
Corresponding min(DI, 1/DI) value: 0.5521
Corresponding statistical parity difference value: -0.1944
Corresponding equal opportunity difference value: 0.0644
Corresponding equal odds difference value: 0.0966
Corresponding average odds difference value: -0.0161
Corresponding Theil index value: 0.0865
race:
Threshold corresponding to Best balanced accuracy: 0.2200
Best balanced accuracy: 0.8091
acc value: 0.7960
auroc value: 0.8091
auprc value: 0.7191
Corresponding min(DI, 1/DI) value: 0.7767
Corresponding statistical parity difference value: -0.0856
Corresponding equal opportunity difference value: 0.0424
Corresponding equal odds difference value: 0.0424
C

In [ ]:
# Using Standard Reweighing Method in series - Race first, followed by Sex
rw_sex = Reweighing(unprivileged_groups=unprivileged_groups1,
                    privileged_groups=privileged_groups1)
rw_race = Reweighing(unprivileged_groups=unprivileged_groups2,
                     privileged_groups=privileged_groups2)
trans_race_dataset = rw_race.fit(
    dataset).transform(dataset)
trans_race_sex_dataset = rw_sex.fit(trans_race_dataset).transform(trans_race_dataset)


trans_race_dataset_train = rw_race.fit(dataset_orig_train).transform(dataset_orig_train)
trans_race_sex_dataset_train = rw_sex.fit(
    trans_race_dataset_train).transform(trans_race_dataset_train)

trans_sex_metric2 = BinaryLabelDatasetMetric(
    dataset=trans_race_sex_dataset, unprivileged_groups=unprivileged_groups1, privileged_groups=privileged_groups1)
trans_race_metric2 = BinaryLabelDatasetMetric(
    dataset=trans_race_sex_dataset, unprivileged_groups=unprivileged_groups2, privileged_groups=privileged_groups2)

print(
    "after reweighing ,sex disparate impact and spd are "

    + str(trans_sex_metric2.disparate_impact())+" "
    + str(trans_sex_metric2.statistical_parity_difference())
)


print(
    "after reweighing ,race disparate impact and spd are "

    + str(trans_race_metric2.disparate_impact())+" "
    + str(trans_race_metric2.statistical_parity_difference())
)

model = make_pipeline(StandardScaler(),
                      LogisticRegression(solver='liblinear', random_state=random_seed))
fit_params = {
    'logisticregression__sample_weight': trans_race_sex_dataset_train.instance_weights}

lr_rw = model.fit(trans_race_sex_dataset_train.features,
                    trans_race_sex_dataset_train.labels.ravel(), **fit_params)
thresh_arr = np.linspace(0.01, 0.5, 50)

print("sex:")
SeriesRW2_sex_val_metrics = test(dataset=dataset_orig_val,
                   model=lr_rw,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups1, privileged_groups=privileged_groups1)

describe_metrics(SeriesRW2_sex_val_metrics, thresh_arr)
print("race:")
SeriesRW2_race_val_metrics = test(dataset=dataset_orig_val,
                   model=lr_rw,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups2, privileged_groups=privileged_groups2)

describe_metrics(SeriesRW2_race_val_metrics, thresh_arr)


after reweighing ,sex disparate impact and spd are 1.0 0.0
after reweighing ,race disparate impact and spd are 1.0949907626554445 0.023234259897132292
sex:
Threshold corresponding to Best balanced accuracy: 0.2200
Best balanced accuracy: 0.8092
acc value: 0.7960
auroc value: 0.8092
auprc value: 0.7192
Corresponding min(DI, 1/DI) value: 0.5519
Corresponding statistical parity difference value: -0.1947
Corresponding equal opportunity difference value: 0.0617
Corresponding equal odds difference value: 0.0963
Corresponding average odds difference value: -0.0173
Corresponding Theil index value: 0.0864
race:
Threshold corresponding to Best balanced accuracy: 0.2200
Best balanced accuracy: 0.8092
acc value: 0.7960
auroc value: 0.8092
auprc value: 0.7192
Corresponding min(DI, 1/DI) value: 0.8042
Corresponding statistical parity difference value: -0.0748
Corresponding equal opportunity difference value: 0.0566
Corresponding equal odds difference value: 0.0566
Corresponding average odds differen

In [ ]:
# Result Compilation
results_data = []

# Original
results_data.append({
    'Model': 'Original',
    'Protected Attribute': 'Sex',
    'Accuracy' : ori_sex_val_metrics['acc'][np.argmax(ori_sex_val_metrics['bal_acc'])],
    'Balanced Accuracy': ori_sex_val_metrics['bal_acc'][np.argmax(ori_sex_val_metrics['bal_acc'])],
    'Disparate Impact': ori_sex.disparate_impact(),
    'Statistical Parity Difference': ori_sex.statistical_parity_difference(),
    'Equal Opportunity Difference': ori_sex_val_metrics['eq_opp_diff'][np.argmax(ori_sex_val_metrics['bal_acc'])],
    'Equalized Odds Difference': ori_sex_val_metrics['eq_odd_diff'][np.argmax(ori_sex_val_metrics['bal_acc'])],
    'Average Odds Difference': ori_sex_val_metrics['avg_odds_diff'][np.argmax(ori_sex_val_metrics['bal_acc'])]
})
results_data.append({
    'Model': 'Original',
    'Protected Attribute': 'Race',
    'Accuracy' : ori_sex_val_metrics['acc'][np.argmax(ori_sex_val_metrics['bal_acc'])],
    'Balanced Accuracy': ori_race_val_metrics['bal_acc'][np.argmax(ori_race_val_metrics['bal_acc'])],
    'Disparate Impact': ori_race.disparate_impact(),
    'Statistical Parity Difference': ori_race.statistical_parity_difference(),
    'Equal Opportunity Difference': ori_race_val_metrics['eq_opp_diff'][np.argmax(ori_race_val_metrics['bal_acc'])],
    'Equalized Odds Difference': ori_race_val_metrics['eq_odd_diff'][np.argmax(ori_race_val_metrics['bal_acc'])],
    'Average Odds Difference': ori_race_val_metrics['avg_odds_diff'][np.argmax(ori_race_val_metrics['bal_acc'])]
})

# MMRW
results_data.append({
    'Model': 'MMRW',
    'Protected Attribute': 'Sex',
    'Accuracy' : mmrw_sex_val_metrics['acc'][np.argmax(mmrw_sex_val_metrics['bal_acc'])],
    'Balanced Accuracy': mmrw_sex_val_metrics['bal_acc'][np.argmax(mmrw_sex_val_metrics['bal_acc'])],
    'Disparate Impact': mmrw_trans_sex.disparate_impact(),
    'Statistical Parity Difference': mmrw_trans_sex.statistical_parity_difference(),
    'Equal Opportunity Difference': mmrw_sex_val_metrics['eq_opp_diff'][np.argmax(mmrw_sex_val_metrics['bal_acc'])],
    'Equalized Odds Difference': mmrw_sex_val_metrics['eq_odd_diff'][np.argmax(mmrw_sex_val_metrics['bal_acc'])],
    'Average Odds Difference': mmrw_sex_val_metrics['avg_odds_diff'][np.argmax(mmrw_sex_val_metrics['bal_acc'])]

})
results_data.append({
    'Model': 'MMRW',
    'Protected Attribute': 'Race',
    'Accuracy' : mmrw_race_val_metrics['acc'][np.argmax(mmrw_race_val_metrics['bal_acc'])],
    'Balanced Accuracy': mmrw_race_val_metrics['bal_acc'][np.argmax(mmrw_race_val_metrics['bal_acc'])],
    'Disparate Impact': mmrw_trans_race.disparate_impact(),
    'Statistical Parity Difference': mmrw_trans_race.statistical_parity_difference(),
    'Equal Opportunity Difference': mmrw_race_val_metrics['eq_opp_diff'][np.argmax(mmrw_race_val_metrics['bal_acc'])],
    'Equalized Odds Difference': mmrw_race_val_metrics['eq_odd_diff'][np.argmax(mmrw_race_val_metrics['bal_acc'])],
    'Average Odds Difference': mmrw_race_val_metrics['avg_odds_diff'][np.argmax(mmrw_race_val_metrics['bal_acc'])]
})

# RW in Series (Sex and Race)
results_data.append({
    'Model': 'RW Series (Sex->Race)',
    'Protected Attribute': 'Sex',
    'Accuracy' : SeriesRW1_sex_val_metrics['acc'][np.argmax(SeriesRW1_sex_val_metrics['bal_acc'])],
    'Balanced Accuracy': SeriesRW1_sex_val_metrics['bal_acc'][np.argmax(SeriesRW1_sex_val_metrics['bal_acc'])],
    'Disparate Impact': trans_sex_metric.disparate_impact(),
    'Statistical Parity Difference': trans_sex_metric.statistical_parity_difference(),
    'Equal Opportunity Difference': SeriesRW1_sex_val_metrics['eq_opp_diff'][np.argmax(SeriesRW1_sex_val_metrics['bal_acc'])],
    'Equalized Odds Difference': SeriesRW1_sex_val_metrics['eq_odd_diff'][np.argmax(SeriesRW1_sex_val_metrics['bal_acc'])],
    'Average Odds Difference': SeriesRW1_sex_val_metrics['avg_odds_diff'][np.argmax(SeriesRW1_sex_val_metrics['bal_acc'])]
})
results_data.append({
    'Model': 'RW Series (Sex->Race)',
    'Protected Attribute': 'Race',
    'Accuracy': SeriesRW1_race_val_metrics['acc'][np.argmax(SeriesRW1_race_val_metrics['bal_acc'])],
    'Balanced Accuracy': SeriesRW1_race_val_metrics['bal_acc'][np.argmax(SeriesRW1_race_val_metrics['bal_acc'])],
    'Disparate Impact': trans_race_metric.disparate_impact(),
    'Statistical Parity Difference': trans_race_metric.statistical_parity_difference(),
    'Equal Opportunity Difference': SeriesRW1_race_val_metrics['eq_opp_diff'][np.argmax(SeriesRW1_race_val_metrics['bal_acc'])],
    'Equalized Odds Difference': SeriesRW1_race_val_metrics['eq_odd_diff'][np.argmax(SeriesRW1_race_val_metrics['bal_acc'])],
    'Average Odds Difference': SeriesRW1_race_val_metrics['avg_odds_diff'][np.argmax(SeriesRW1_race_val_metrics['bal_acc'])]
})

# RW in Series (Race and Sex)
results_data.append({
    'Model': 'RW Series (Race->Sex)',
    'Protected Attribute': 'Sex',
    'Accuracy': SeriesRW2_sex_val_metrics['acc'][np.argmax(SeriesRW2_sex_val_metrics['bal_acc'])],
    'Balanced Accuracy': SeriesRW2_sex_val_metrics['bal_acc'][np.argmax(SeriesRW2_sex_val_metrics['bal_acc'])],
    'Disparate Impact': trans_sex_metric2.disparate_impact(),
    'Statistical Parity Difference': trans_sex_metric2.statistical_parity_difference(),
    'Equal Opportunity Difference': SeriesRW2_sex_val_metrics['eq_opp_diff'][np.argmax(SeriesRW2_sex_val_metrics['bal_acc'])],
    'Equalized Odds Difference': SeriesRW2_sex_val_metrics['eq_odd_diff'][np.argmax(SeriesRW2_sex_val_metrics['bal_acc'])],
    'Average Odds Difference': SeriesRW2_sex_val_metrics['avg_odds_diff'][np.argmax(SeriesRW2_sex_val_metrics['bal_acc'])]
})
results_data.append({
    'Model': 'RW Series (Race->Sex)',
    'Protected Attribute': 'Race',
    'Accuracy': SeriesRW2_race_val_metrics['acc'][np.argmax(SeriesRW2_race_val_metrics['bal_acc'])],
    'Balanced Accuracy': SeriesRW2_race_val_metrics['bal_acc'][np.argmax(SeriesRW2_race_val_metrics['bal_acc'])],
    'Disparate Impact': trans_race_metric2.disparate_impact(),
    'Statistical Parity Difference': trans_race_metric2.statistical_parity_difference(),
    'Equal Opportunity Difference': SeriesRW2_race_val_metrics['eq_opp_diff'][np.argmax(SeriesRW2_race_val_metrics['bal_acc'])],
    'Equalized Odds Difference': SeriesRW2_race_val_metrics['eq_odd_diff'][np.argmax(SeriesRW2_race_val_metrics['bal_acc'])],
    'Average Odds Difference': SeriesRW2_race_val_metrics['avg_odds_diff'][np.argmax(SeriesRW2_race_val_metrics['bal_acc'])]
})

results_df = pd.DataFrame(results_data)
display(results_df)

# Create a pivot table for the heatmap
heatmap_data = results_df.pivot_table(index=['Model', 'Protected Attribute'], values=[
    'Accuracy',
    'Balanced Accuracy',
    'Disparate Impact',
    'Statistical Parity Difference',
    'Equal Opportunity Difference',
    'Equalized Odds Difference',
    'Average Odds Difference'
])


,Model,Protected Attribute,Accuracy,Balanced Accuracy,Disparate Impact,Statistical Parity Difference,Equal Opportunity Difference,Equalized Odds Difference,Average Odds Difference
0,Original,Sex,0.791184,0.816067,0.363470,-1.989014e-01,-0.133100,0.249815,-0.191458
1,Original,Race,0.791184,0.816067,0.603769,-1.039594e-01,-0.056920,0.109998,-0.083459
2,MMRW,Sex,0.795828,0.809430,1.000000,0.000000e+00,0.062717,0.101199,-0.019241
3,MMRW,Race,0.795828,0.809430,1.000000,-2.775558e-17,0.037099,0.037099,0.000320
4,RW Series (Sex->Race),Sex,0.796049,0.809085,1.021865,5.380859e-03,0.064442,0.096595,-0.016077
5,RW Series (Sex->Race),Race,0.796049,0.809085,1.000000,-5.551115e-17,0.042363,0.042363,0.005264
6,RW Series (Race->Sex),Sex,0.796049,0.809184,1.000000,0.000000e+00,0.061719,0.096338,-0.017309
7,RW Series (Race->Sex),Race,0.796049,0.809184,1.094991,2.323426e-02,0.056622,0.056622,0.017503


In [ ]:
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

# Function to highlight the min values (closest to zero) in the specified columns
def highlight_min_abs(s):
    is_min_abs = abs(s) == abs(s).min()
    return ['background-color: yellow' if v else '' for v in is_min_abs]

In [ ]:
results_df.style.apply(
    highlight_max, subset=['Accuracy', 'Balanced Accuracy', 'Disparate Impact']).apply(
        highlight_min_abs,subset=['Statistical Parity Difference',
                                  'Equal Opportunity Difference',
                                  'Equalized Odds Difference',
                                  'Average Odds Difference'])

,Model,Protected Attribute,Accuracy,Balanced Accuracy,Disparate Impact,Statistical Parity Difference,Equal Opportunity Difference,Equalized Odds Difference,Average Odds Difference
0,Original,Sex,0.791184,0.816067,0.363470,-0.198901,-0.133100,0.249815,-0.191458
1,Original,Race,0.791184,0.816067,0.603769,-0.103959,-0.056920,0.109998,-0.083459
2,MMRW,Sex,0.839169,0.809430,1.000000,0.000000,0.062717,0.101199,-0.019241
3,MMRW,Race,0.839169,0.809430,1.000000,-0.000000,0.037099,0.037099,0.000320
4,RW Series (Sex->Race),Sex,0.839169,0.809085,1.021865,0.005381,0.064442,0.096595,-0.016077
5,RW Series (Sex->Race),Race,0.839169,0.809085,1.000000,-0.000000,0.042363,0.042363,0.005264
6,RW Series (Race->Sex),Sex,0.838947,0.809184,1.000000,0.000000,0.061719,0.096338,-0.017309
7,RW Series (Race->Sex),Race,0.838947,0.809184,1.094991,0.023234,0.056622,0.056622,0.017503


In [ ]:
def highlight_improvements_with_shade(row):
    # Get the corresponding original row for comparison
    original_sex_row = results_df[(results_df['Model'] == 'Original') & (results_df['Protected Attribute'] == 'Sex')].iloc[0]
    original_race_row = results_df[(results_df['Model'] == 'Original') & (results_df['Protected Attribute'] == 'Race')].iloc[0]

    styles = [''] * len(row)

    # Determine the original row to compare against
    if row['Protected Attribute'] == 'Sex':
        original_row = original_sex_row
    else:
        original_row = original_race_row

    # Define metrics where higher is better (excluding Disparate Impact where closer to 1 is better)
    higher_is_better = ['Accuracy', 'Balanced Accuracy']
    # Define metrics where absolute value closer to 0 is better
    closer_to_zero_is_better = ['Statistical Parity Difference', 'Equal Opportunity Difference', 'Equalized Odds Difference', 'Average Odds Difference']
    # Disparate Impact is better when closer to 1
    disparate_impact_col = 'Disparate Impact'

    for i, col in enumerate(results_df.columns):
        if col in higher_is_better:
            original_value = original_row[col]
            current_value = row[col]
            if current_value > original_value:
                # Calculate the improvement as a percentage of the original value (handle division by zero)
                improvement = (current_value - original_value) / original_value if original_value != 0 else (current_value - original_value)
                # Map improvement to a shade (e.g., darker green for larger improvement)
                # This is a simple linear mapping, can be adjusted
                shade = min(255, int(improvement * 100 * 3)) # Scale factor, adjust as needed
                styles[i] = f'background-color: rgb({255-shade},{255},{255-shade})' # Shades of green

        elif col in closer_to_zero_is_better:
            original_abs = abs(original_row[col])
            current_abs = abs(row[col])
            if current_abs < original_abs:
                 # Calculate the improvement as the reduction in absolute value
                improvement = original_abs - current_abs
                # Map improvement to a shade
                shade = min(255, int(improvement * 100 * 3)) # Scale factor, adjust as needed
                styles[i] = f'background-color: rgb({255-shade},{255},{255-shade})' # Shades of green

        elif col == disparate_impact_col and row['Model'] != 'Original': # Only compare DI for mitigated models
            original_diff_from_one = abs(original_row[col] - 1)
            current_diff_from_one = abs(row[col] - 1)
            if current_diff_from_one < original_diff_from_one:
                # Calculate improvement based on closeness to 1
                improvement = original_diff_from_one - current_diff_from_one
                # Map improvement to a shade
                shade = min(255, int(improvement * 100 * 3)) # Scale factor, adjust as needed
                styles[i] = f'background-color: rgb({255-shade},{255},{255-shade})' # Shades of green


    return styles

# Apply the highlighting function to the DataFrame
styled_results_df = results_df.style.apply(highlight_improvements_with_shade, axis=1)
display(styled_results_df)

,Model,Protected Attribute,Accuracy,Balanced Accuracy,Disparate Impact,Statistical Parity Difference,Equal Opportunity Difference,Equalized Odds Difference,Average Odds Difference
0,Original,Sex,0.791184,0.816067,0.363470,-0.198901,-0.133100,0.249815,-0.191458
1,Original,Race,0.791184,0.816067,0.603769,-0.103959,-0.056920,0.109998,-0.083459
2,MMRW,Sex,0.839169,0.809430,1.000000,0.000000,0.062717,0.101199,-0.019241
3,MMRW,Race,0.839169,0.809430,1.000000,-0.000000,0.037099,0.037099,0.000320
4,RW Series (Sex->Race),Sex,0.839169,0.809085,1.021865,0.005381,0.064442,0.096595,-0.016077
5,RW Series (Sex->Race),Race,0.839169,0.809085,1.000000,-0.000000,0.042363,0.042363,0.005264
6,RW Series (Race->Sex),Sex,0.838947,0.809184,1.000000,0.000000,0.061719,0.096338,-0.017309
7,RW Series (Race->Sex),Race,0.838947,0.809184,1.094991,0.023234,0.056622,0.056622,0.017503


In [ ]:
# Export the results DataFrame to a CSV file
results_df.to_csv('/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/Results/MMRW_results_Adult_2000.csv', index=False)

print("results_df exported to your Drive folder.")

results_df exported to your Drive folder.


# Adult

In [6]:
def run_mmrw_fairness_pipeline(dataset,
                              sensitive_features,
                              multi_privileged_groups,
                              multi_unprivileged_groups,
                              privileged_groups_dict,
                              unprivileged_groups_dict,
                              output_csv_path):


    # Split the dataset
    dataset_orig_train, dataset_orig_val = dataset.split([0.7], shuffle=True, seed=random_seed)

    thresh_arr = np.linspace(0.01, 0.5, 50)

    def get_metric_results(dataset, model, val_dataset, unprivileged_groups, privileged_groups):
        val_metrics = test(dataset=val_dataset, model=model, thresh_arr=thresh_arr,
                           unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        best_idx = np.argmax(val_metrics['bal_acc'])
        return {
            'Accuracy': val_metrics['acc'][best_idx],
            'Balanced Accuracy': val_metrics['bal_acc'][best_idx],
            'Equal Opportunity Difference': val_metrics['eq_opp_diff'][best_idx],
            'Equalized Odds Difference': val_metrics['eq_odd_diff'][best_idx],
            'Average Odds Difference': val_metrics['avg_odds_diff'][best_idx]
        }

    def train_model(X, y, sample_weight):
        model = make_pipeline(StandardScaler(),
                              LogisticRegression(solver='liblinear', random_state=random_seed))
        fit_params = {'logisticregression__sample_weight': sample_weight}
        model.fit(X, y.ravel(), **fit_params)
        return model

    results_data = []

    for method in ['original', 'mmrw', 'rw_sex_race', 'rw_race_sex']:
        if method == 'original':
            model = train_model(dataset_orig_train.features, dataset_orig_train.labels,
                                dataset_orig_train.instance_weights)
            transformed_dataset = dataset
        elif method == 'mmrw':
            mmrw = MultiLevelReweighing(multi_unprivileged_groups, multi_privileged_groups)
            transformed_dataset = mmrw.fit(dataset).transform(dataset)
            transformed_train = mmrw.fit(dataset_orig_train).transform(dataset_orig_train)
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)
        elif method == 'rw_sex_race':
            rw1 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[0]],
                             privileged_groups=privileged_groups_dict[sensitive_features[0]])
            rw2 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[1]],
                             privileged_groups=privileged_groups_dict[sensitive_features[1]])
            transformed_dataset = rw2.fit(rw1.fit(dataset).transform(dataset)).transform(
                rw1.fit(dataset).transform(dataset))
            transformed_train = rw2.fit(rw1.fit(dataset_orig_train).transform(dataset_orig_train)).transform(
                rw1.fit(dataset_orig_train).transform(dataset_orig_train))
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)
        elif method == 'rw_race_sex':
            rw1 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[1]],
                             privileged_groups=privileged_groups_dict[sensitive_features[1]])
            rw2 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[0]],
                             privileged_groups=privileged_groups_dict[sensitive_features[0]])
            transformed_dataset = rw2.fit(rw1.fit(dataset).transform(dataset)).transform(
                rw1.fit(dataset).transform(dataset))
            transformed_train = rw2.fit(rw1.fit(dataset_orig_train).transform(dataset_orig_train)).transform(
                rw1.fit(dataset_orig_train).transform(dataset_orig_train))
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)

        for attr in sensitive_features:
            metric = BinaryLabelDatasetMetric(dataset=transformed_dataset,
                                              unprivileged_groups=unprivileged_groups_dict[attr],
                                              privileged_groups=privileged_groups_dict[attr])
            metric_result = get_metric_results(dataset_orig_val, model, dataset_orig_val,
                                               unprivileged_groups_dict[attr],
                                               privileged_groups_dict[attr])

            results_data.append({
                'Model': method.upper(),
                'Protected Attribute': attr.capitalize(),
                'Accuracy': metric_result['Accuracy'],
                'Balanced Accuracy': metric_result['Balanced Accuracy'],
                'Disparate Impact': metric.disparate_impact(),
                'Statistical Parity Difference': metric.statistical_parity_difference(),
                'Equal Opportunity Difference': metric_result['Equal Opportunity Difference'],
                'Equalized Odds Difference': metric_result['Equalized Odds Difference'],
                'Average Odds Difference': metric_result['Average Odds Difference']
            })

    results_df = pd.DataFrame(results_data)
    results_df.to_csv(output_csv_path, index=False)
    return results_df


In [7]:
def full_run(data_path, result_csv_path):
    # --- Step 1: Load and Prepare Data ---
    df = pd.read_csv(data_path)
    df.columns = df.columns.str.strip().str.lower()

    # List of potential categorical features
    potential_categorical_features = [
        'workclass', 'education', 'marital-status',
        'occupation', 'relationship', 'native-country'
    ]

    categorical_features = [
        col for col in potential_categorical_features
        if col in df.columns and df[col].dtype == 'object'
    ]

    # Ensure empty list if no categorical features found
    categorical_features = categorical_features or []

    # Now safely build the StandardDataset
    dataset = StandardDataset(
        df=df,
        label_name='income',
        favorable_classes=[1],
        protected_attribute_names=['sex', 'race'],
        privileged_classes=[[1], [1]],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=[],
        features_to_drop=[],
        na_values=['?']
    )

    # --- Step 2: Define Protected Groups ---
    protected_attributes = ['sex', 'race']
    multi_privileged_groups = [
        {"feature_name": "race", "privileged_value": 1, "level": 1},
        {"feature_name": "sex", "privileged_value": 1, "level": 2},
    ]
    multi_unprivileged_groups = [
        {"feature_name": "race", "unprivileged_value": 0, "level": 1},
        {"feature_name": "sex", "unprivileged_value": 0, "level": 2},
    ]
    privileged_groups_dict = {
        'sex': [{"sex": 1}],
        'race': [{"race": 1}]
    }
    unprivileged_groups_dict = {
        'sex': [{"sex": 0}],
        'race': [{"race": 0}]
    }

    # --- Step 3: Run full fairness pipeline ---
    results_df = run_mmrw_fairness_pipeline(
        dataset=dataset,
        sensitive_features=protected_attributes,
        multi_privileged_groups=multi_privileged_groups,
        multi_unprivileged_groups=multi_unprivileged_groups,
        privileged_groups_dict=privileged_groups_dict,
        unprivileged_groups_dict=unprivileged_groups_dict,
        output_csv_path=result_csv_path
    )

    return results_df


In [ ]:
results_df = full_run(
    data_path="/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Raw_Race/",
    result_csv_path="/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/Results/Adult/MMRW_results_Adult_orig.csv"
)

results_df

KeyError: 'education'

In [10]:
# Define paths
data_folder = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Raw_Race"
result_folder = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/Results/Adult"

# Get all CSV files in the data folder
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))

# Store results
all_results = []

for file_path in csv_files:
    filename = os.path.basename(file_path).replace(".csv", "")
    result_csv_path = os.path.join(result_folder, f"MMRW_results_{filename}.csv")

    print(f"📂 Processing: {filename}")
    results_df = full_run(data_path=file_path, result_csv_path=result_csv_path)

    results_df["dataset"] = filename  # Optional: tag results with file name
    all_results.append(results_df)

# Concatenate all into a single DataFrame if needed
combined_results_df = pd.concat(all_results, axis=0).reset_index(drop=True)

# Save combined summary
combined_results_df.to_csv(os.path.join(result_folder, "AdultRace_MMRW_results_combined_summary.csv"), index=False)

# Show final results
combined_results_df

📂 Processing: Adult_Race_bias3
📂 Processing: Adult_Race_bias1
📂 Processing: Adult_Race_bias2
📂 Processing: Adult_Race_bias4
📂 Processing: Adult_Race_GenAlgo


/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFin

📂 Processing: Adult_Census_orig
📂 Processing: Adult_Census_2015
📂 Processing: Adult_Census_2010
📂 Processing: Adult_Census_2005
📂 Processing: Adult_Census_2000


,Model,Protected Attribute,Accuracy,Balanced Accuracy,Disparate Impact,Statistical Parity Difference,Equal Opportunity Difference,Equalized Odds Difference,Average Odds Difference,dataset
0,ORIGINAL,Sex,0.802978,0.806814,0.525820,-1.681461e-01,-0.036904,0.194442,-0.115673,Adult_Race_bias3
1,ORIGINAL,Race,0.802978,0.806814,1.522852,1.461588e-01,0.067421,0.076430,0.071925,Adult_Race_bias3
2,MMRW,Sex,0.794796,0.803136,1.000000,1.665335e-16,0.077680,0.102900,-0.012610,Adult_Race_bias3
3,MMRW,Race,0.794796,0.803136,1.000000,0.000000e+00,-0.005166,0.033432,-0.019299,Adult_Race_bias3
4,RW_SEX_RACE,Sex,0.796713,0.803620,0.913393,-2.672074e-02,0.064739,0.113855,-0.024558,Adult_Race_bias3
...,...,...,...,...,...,...,...,...,...,...
75,MMRW,Race,0.751200,0.782194,1.000000,5.551115e-17,-0.004766,0.013952,-0.009359,Adult_Census_2000
76,RW_SEX_RACE,Sex,0.751000,0.781839,1.010406,1.250693e-03,0.010090,0.028831,0.019461,Adult_Census_2000
77,RW_SEX_RACE,Race,0.751000,0.781839,1.000000,-1.387779e-17,-0.004140,0.012276,-0.008208,Adult_Census_2000
78,RW_RACE_SEX,Sex,0.779867,0.782532,1.000000,-5.551115e-17,0.004513,0.025102,0.014807,Adult_Census_2000


# German

In [ ]:
def run_mmrw_fairness_pipeline(dataset,
                              sensitive_features,
                              multi_privileged_groups,
                              multi_unprivileged_groups,
                              privileged_groups_dict,
                              unprivileged_groups_dict,
                              output_csv_path):


    # Split the dataset
    dataset_orig_train, dataset_orig_val = dataset.split([0.7], shuffle=True, seed=random_seed)

    thresh_arr = np.linspace(0.01, 0.5, 50)

    def get_metric_results(dataset, model, val_dataset, unprivileged_groups, privileged_groups):
        val_metrics = test(dataset=val_dataset, model=model, thresh_arr=thresh_arr,
                           unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        best_idx = np.argmax(val_metrics['bal_acc'])
        return {
            'Accuracy': val_metrics['acc'][best_idx],
            'Balanced Accuracy': val_metrics['bal_acc'][best_idx],
            'Equal Opportunity Difference': val_metrics['eq_opp_diff'][best_idx],
            'Equalized Odds Difference': val_metrics['eq_odd_diff'][best_idx],
            'Average Odds Difference': val_metrics['avg_odds_diff'][best_idx]
        }

    def train_model(X, y, sample_weight):
        model = make_pipeline(StandardScaler(),
                              LogisticRegression(solver='liblinear', random_state=random_seed))
        fit_params = {'logisticregression__sample_weight': sample_weight}
        model.fit(X, y.ravel(), **fit_params)
        return model

    results_data = []

    for method in ['original', 'mmrw', 'rw_sex_age', 'rw_age_sex']:
        if method == 'original':
            model = train_model(dataset_orig_train.features, dataset_orig_train.labels,
                                dataset_orig_train.instance_weights)
            transformed_dataset = dataset
        elif method == 'mmrw':
            mmrw = MultiLevelReweighing(multi_unprivileged_groups, multi_privileged_groups)
            transformed_dataset = mmrw.fit(dataset).transform(dataset)
            transformed_train = mmrw.fit(dataset_orig_train).transform(dataset_orig_train)
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)
        elif method == 'rw_sex_age':
            rw1 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[0]],
                             privileged_groups=privileged_groups_dict[sensitive_features[0]])
            rw2 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[1]],
                             privileged_groups=privileged_groups_dict[sensitive_features[1]])
            transformed_dataset = rw2.fit(rw1.fit(dataset).transform(dataset)).transform(
                rw1.fit(dataset).transform(dataset))
            transformed_train = rw2.fit(rw1.fit(dataset_orig_train).transform(dataset_orig_train)).transform(
                rw1.fit(dataset_orig_train).transform(dataset_orig_train))
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)
        elif method == 'rw_age_sex':
            rw1 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[1]],
                             privileged_groups=privileged_groups_dict[sensitive_features[1]])
            rw2 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[0]],
                             privileged_groups=privileged_groups_dict[sensitive_features[0]])
            transformed_dataset = rw2.fit(rw1.fit(dataset).transform(dataset)).transform(
                rw1.fit(dataset).transform(dataset))
            transformed_train = rw2.fit(rw1.fit(dataset_orig_train).transform(dataset_orig_train)).transform(
                rw1.fit(dataset_orig_train).transform(dataset_orig_train))
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)

        for attr in sensitive_features:
            metric = BinaryLabelDatasetMetric(dataset=transformed_dataset,
                                              unprivileged_groups=unprivileged_groups_dict[attr],
                                              privileged_groups=privileged_groups_dict[attr])
            metric_result = get_metric_results(dataset_orig_val, model, dataset_orig_val,
                                               unprivileged_groups_dict[attr],
                                               privileged_groups_dict[attr])

            results_data.append({
                'Model': method.upper(),
                'Protected Attribute': attr.capitalize(),
                'Accuracy': metric_result['Accuracy'],
                'Balanced Accuracy': metric_result['Balanced Accuracy'],
                'Disparate Impact': metric.disparate_impact(),
                'Statistical Parity Difference': metric.statistical_parity_difference(),
                'Equal Opportunity Difference': metric_result['Equal Opportunity Difference'],
                'Equalized Odds Difference': metric_result['Equalized Odds Difference'],
                'Average Odds Difference': metric_result['Average Odds Difference']
            })

    results_df = pd.DataFrame(results_data)
    results_df.to_csv(output_csv_path, index=False)
    return results_df


In [ ]:
def full_run(data_path, result_csv_path):
    # --- Step 1: Load and Prepare Data ---
    df = pd.read_csv(data_path)
    df.columns = df.columns.str.strip().str.lower()

    # List of potential categorical features
    potential_categorical_features = [
        'status', 'credit_history', 'purpose', 'savings',
        'employment', 'other_debtors', 'property',
        'other_installment_plans', 'housing',
        'job', 'telephone', 'foreign_worker'
    ]

    categorical_features = [
        col for col in potential_categorical_features
        if col in df.columns and df[col].dtype == 'object'
    ]

    # Ensure empty list if no categorical features found
    categorical_features = categorical_features or []

    # Now safely build the StandardDataset
    dataset = StandardDataset(
        df=df,
        label_name='credit_risk',
        favorable_classes=[1],
        protected_attribute_names=['sex', 'age'],
        privileged_classes=[[1], [1]],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=[],
        features_to_drop=[],
        na_values=['?']
    )

    # --- Step 2: Define Protected Groups ---
    protected_attributes = ['sex', 'age']
    multi_privileged_groups = [
        {"feature_name": "age", "privileged_value": 1, "level": 1},
        {"feature_name": "sex", "privileged_value": 1, "level": 2},
    ]
    multi_unprivileged_groups = [
        {"feature_name": "age", "unprivileged_value": 0, "level": 1},
        {"feature_name": "sex", "unprivileged_value": 0, "level": 2},
    ]
    privileged_groups_dict = {
        'sex': [{"sex": 1}],
        'age': [{"age": 1}]
    }
    unprivileged_groups_dict = {
        'sex': [{"sex": 0}],
        'age': [{"age": 0}]
    }

    # --- Step 3: Run full fairness pipeline ---
    results_df = run_mmrw_fairness_pipeline(
        dataset=dataset,
        sensitive_features=protected_attributes,
        multi_privileged_groups=multi_privileged_groups,
        multi_unprivileged_groups=multi_unprivileged_groups,
        privileged_groups_dict=privileged_groups_dict,
        unprivileged_groups_dict=unprivileged_groups_dict,
        output_csv_path=result_csv_path
    )

    return results_df


In [ ]:
# Define paths
data_folder = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/German/Raw"
result_folder = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/Results/German"

# Get all CSV files in the data folder
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))

# Store results
all_results = []

for file_path in csv_files:
    filename = os.path.basename(file_path).replace(".csv", "")
    result_csv_path = os.path.join(result_folder, f"MMRW_results_{filename}.csv")

    print(f"Processing: {filename}")
    results_df = full_run(data_path=file_path, result_csv_path=result_csv_path)

    results_df["dataset"] = filename  # Optional: tag results with file name
    all_results.append(results_df)

# Concatenate all into a single DataFrame if needed
combined_results_df = pd.concat(all_results, axis=0).reset_index(drop=True)

# Save combined summary
combined_results_df.to_csv(os.path.join(result_folder, "MMRW_results_combined_summary.csv"), index=False)

# Show final results
combined_results_df

📂 Processing: German_orig


/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFin

📂 Processing: German_bias1


/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFin

📂 Processing: German_bias2


...

📂 Processing: German_GenAlgo_age


/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFin

📂 Processing: German_GenAlgo_sex


/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFin

,Model,Protected Attribute,Accuracy,Balanced Accuracy,Disparate Impact,Statistical Parity Difference,Equal Opportunity Difference,Equalized Odds Difference,Average Odds Difference,dataset
0,ORIGINAL,Sex,0.756667,0.697124,0.896567,-7.480131e-02,-0.048789,0.271512,-0.160151,German_orig
1,ORIGINAL,Age,0.756667,0.697124,0.794826,-1.494477e-01,-0.181744,0.358216,-0.269980,German_orig
2,MMRW,Sex,0.740000,0.665058,1.000000,-1.110223e-16,0.002935,0.060986,-0.029026,German_orig
3,MMRW,Age,0.740000,0.665058,1.000000,2.220446e-16,0.002434,0.044131,-0.020849,German_orig
4,RW_SEX_AGE,Sex,0.740000,0.665058,1.037955,2.625925e-02,0.002935,0.060986,-0.029026,German_orig
...,...,...,...,...,...,...,...,...,...,...
83,MMRW,Age,0.786667,0.799020,1.000000,-2.220446e-16,0.077778,0.168575,0.123177,German_GenAlgo_sex
84,RW_SEX_AGE,Sex,0.786667,0.799020,1.032619,2.228809e-02,-0.049983,0.116152,0.033085,German_GenAlgo_sex
85,RW_SEX_AGE,Age,0.786667,0.799020,1.000000,0.000000e+00,0.077778,0.168575,0.123177,German_GenAlgo_sex
86,RW_AGE_SEX,Sex,0.786667,0.799020,1.000000,-2.220446e-16,-0.049983,0.072595,0.011306,German_GenAlgo_sex


# Student

In [ ]:
def run_mmrw_fairness_pipeline(dataset,
                              sensitive_features,
                              multi_privileged_groups,
                              multi_unprivileged_groups,
                              privileged_groups_dict,
                              unprivileged_groups_dict,
                              output_csv_path):


    # Split the dataset
    dataset_orig_train, dataset_orig_val = dataset.split([0.7], shuffle=True, seed=random_seed)

    thresh_arr = np.linspace(0.01, 0.5, 50)

    def get_metric_results(dataset, model, val_dataset, unprivileged_groups, privileged_groups):
        val_metrics = test(dataset=val_dataset, model=model, thresh_arr=thresh_arr,
                           unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        best_idx = np.argmax(val_metrics['bal_acc'])
        return {
            'Accuracy': val_metrics['acc'][best_idx],
            'Balanced Accuracy': val_metrics['bal_acc'][best_idx],
            'Equal Opportunity Difference': val_metrics['eq_opp_diff'][best_idx],
            'Equalized Odds Difference': val_metrics['eq_odd_diff'][best_idx],
            'Average Odds Difference': val_metrics['avg_odds_diff'][best_idx]
        }

    def train_model(X, y, sample_weight):
        model = make_pipeline(StandardScaler(),
                              LogisticRegression(solver='liblinear', random_state=random_seed))
        fit_params = {'logisticregression__sample_weight': sample_weight}
        model.fit(X, y.ravel(), **fit_params)
        return model

    results_data = []

    for method in ['original', 'mmrw', 'rw_sex_age', 'rw_age_sex']:
        if method == 'original':
            model = train_model(dataset_orig_train.features, dataset_orig_train.labels,
                                dataset_orig_train.instance_weights)
            transformed_dataset = dataset
        elif method == 'mmrw':
            mmrw = MultiLevelReweighing(multi_unprivileged_groups, multi_privileged_groups)
            transformed_dataset = mmrw.fit(dataset).transform(dataset)
            transformed_train = mmrw.fit(dataset_orig_train).transform(dataset_orig_train)
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)
        elif method == 'rw_sex_age':
            rw1 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[0]],
                             privileged_groups=privileged_groups_dict[sensitive_features[0]])
            rw2 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[1]],
                             privileged_groups=privileged_groups_dict[sensitive_features[1]])
            transformed_dataset = rw2.fit(rw1.fit(dataset).transform(dataset)).transform(
                rw1.fit(dataset).transform(dataset))
            transformed_train = rw2.fit(rw1.fit(dataset_orig_train).transform(dataset_orig_train)).transform(
                rw1.fit(dataset_orig_train).transform(dataset_orig_train))
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)
        elif method == 'rw_age_sex':
            rw1 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[1]],
                             privileged_groups=privileged_groups_dict[sensitive_features[1]])
            rw2 = Reweighing(unprivileged_groups=unprivileged_groups_dict[sensitive_features[0]],
                             privileged_groups=privileged_groups_dict[sensitive_features[0]])
            transformed_dataset = rw2.fit(rw1.fit(dataset).transform(dataset)).transform(
                rw1.fit(dataset).transform(dataset))
            transformed_train = rw2.fit(rw1.fit(dataset_orig_train).transform(dataset_orig_train)).transform(
                rw1.fit(dataset_orig_train).transform(dataset_orig_train))
            model = train_model(transformed_train.features, transformed_train.labels,
                                transformed_train.instance_weights)

        for attr in sensitive_features:
            metric = BinaryLabelDatasetMetric(dataset=transformed_dataset,
                                              unprivileged_groups=unprivileged_groups_dict[attr],
                                              privileged_groups=privileged_groups_dict[attr])
            metric_result = get_metric_results(dataset_orig_val, model, dataset_orig_val,
                                               unprivileged_groups_dict[attr],
                                               privileged_groups_dict[attr])

            results_data.append({
                'Model': method.upper(),
                'Protected Attribute': attr.capitalize(),
                'Accuracy': metric_result['Accuracy'],
                'Balanced Accuracy': metric_result['Balanced Accuracy'],
                'Disparate Impact': metric.disparate_impact(),
                'Statistical Parity Difference': metric.statistical_parity_difference(),
                'Equal Opportunity Difference': metric_result['Equal Opportunity Difference'],
                'Equalized Odds Difference': metric_result['Equalized Odds Difference'],
                'Average Odds Difference': metric_result['Average Odds Difference']
            })

    results_df = pd.DataFrame(results_data)
    results_df.to_csv(output_csv_path, index=False)
    return results_df


In [ ]:
def full_run(data_path, result_csv_path):
    # --- Step 1: Load and Prepare Data ---
    df = pd.read_csv(data_path)
    df.columns = df.columns.str.strip().str.lower()

    # List of potential categorical features
    potential_categorical_features = []

    categorical_features = [
        col for col in potential_categorical_features
        if col in df.columns and df[col].dtype == 'object'
    ]

    # Ensure empty list if no categorical features found
    categorical_features = categorical_features or []

    # Now safely build the StandardDataset
    dataset = StandardDataset(
        df=df,
        label_name='dropout',
        favorable_classes=[1],
        protected_attribute_names=['gender', 'age'],
        privileged_classes=[[1], [1]],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=[],
        features_to_drop=[],
        na_values=['?']
    )

    # --- Step 2: Define Protected Groups ---
    protected_attributes = ['gender', 'age']
    multi_privileged_groups = [
        {"feature_name": "age", "privileged_value": 1, "level": 1},
        {"feature_name": "gender", "privileged_value": 0, "level": 2},
    ]
    multi_unprivileged_groups = [
        {"feature_name": "age", "unprivileged_value": 0, "level": 1},
        {"feature_name": "gender", "unprivileged_value": 1, "level": 2},
    ]
    privileged_groups_dict = {
        'gender': [{"gender": 0}],
        'age': [{"age": 1}]
    }
    unprivileged_groups_dict = {
        'gender': [{"gender": 1}],
        'age': [{"age": 0}]
    }

    # --- Step 3: Run full fairness pipeline ---
    results_df = run_mmrw_fairness_pipeline(
        dataset=dataset,
        sensitive_features=protected_attributes,
        multi_privileged_groups=multi_privileged_groups,
        multi_unprivileged_groups=multi_unprivileged_groups,
        privileged_groups_dict=privileged_groups_dict,
        unprivileged_groups_dict=unprivileged_groups_dict,
        output_csv_path=result_csv_path
    )

    return results_df


In [ ]:
# Define paths
data_folder = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Student/Raw"
result_folder = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/Results/Student"

# Get all CSV files in the data folder
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))

# Store results
all_results = []

for file_path in csv_files:
    filename = os.path.basename(file_path).replace(".csv", "")
    result_csv_path = os.path.join(result_folder, f"MMRW_results_{filename}.csv")

    print(f"Processing: {filename}")
    results_df = full_run(data_path=file_path, result_csv_path=result_csv_path)

    results_df["dataset"] = filename  # Optional: tag results with file name
    all_results.append(results_df)

# Concatenate all into a single DataFrame if needed
combined_results_df = pd.concat(all_results, axis=0).reset_index(drop=True)

# Save combined summary
combined_results_df.to_csv(os.path.join(result_folder, "MMRW_results_combined_summary.csv"), index=False)

# Show final results
combined_results_df

Processing: Student_orig
Processing: Student_GenAlgo_Sex


/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/metrics/eval_metrics.py:21: RuntimeWarning: invalid value encountered in scalar divide
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
/content/drive/MyDrive/UCL_MSc_AI/UCLFin

Processing: Student_bias2
Processing: Student_bias3
Processing: Student_bias4
Processing: Student_Age_bias1
Processing: Student_Age_bias2
Processing: Student_Age_bias3
Processing: Student_Age_bias4
Processing: Student_GenAlgo_Age
Processing: Student_bias1


,Model,Protected Attribute,Accuracy,Balanced Accuracy,Disparate Impact,Statistical Parity Difference,Equal Opportunity Difference,Equalized Odds Difference,Average Odds Difference,dataset
0,ORIGINAL,Gender,0.884789,0.848912,0.733671,-1.994681e-01,-0.086630,0.086630,-0.083338,Student_orig
1,ORIGINAL,Age,0.884789,0.848912,0.586404,-3.111453e-01,-0.043762,0.260342,-0.152052,Student_orig
2,MMRW,Gender,0.871988,0.827720,1.000000,1.110223e-16,-0.036182,0.036182,-0.022787,Student_orig
3,MMRW,Age,0.871988,0.827720,1.000000,2.220446e-16,0.005556,0.145116,-0.069780,Student_orig
4,RW_SEX_AGE,Gender,0.871988,0.828381,1.028577,1.920423e-02,-0.034617,0.034617,-0.019668,Student_orig
...,...,...,...,...,...,...,...,...,...,...
83,MMRW,Age,0.824548,0.814441,1.000000,2.220446e-16,0.023568,0.093120,-0.034776,Student_bias1
84,RW_SEX_AGE,Gender,0.823042,0.815672,1.050184,2.630222e-02,-0.036215,0.036215,-0.014001,Student_bias1
85,RW_SEX_AGE,Age,0.823042,0.815672,1.000000,-1.110223e-16,0.030773,0.078361,-0.023794,Student_bias1
86,RW_AGE_SEX,Gender,0.827560,0.817492,1.000000,-1.110223e-16,-0.042414,0.042414,-0.026239,Student_bias1
